In [1]:
import yfinance as yf
import pandas as pd

In [16]:
df = yf.download(
    tickers="AAPL",
    interval="1m",
    period='8d'
)

/var/folders/r6/jv1hv35d4rg8crmf3xb99ykc0000gn/T/ipykernel_42185/200115773.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed


In [21]:
df = df.droplevel("Ticker", axis=1)
df

Price,Close,High,Low,Open,Volume
Datetime,,,,,
2025-09-05 13:30:00+00:00,239.699997,240.000000,239.410004,239.990005,3861130
2025-09-05 13:31:00+00:00,239.580002,240.429993,239.520004,239.699997,986618
2025-09-05 13:32:00+00:00,239.960007,239.974594,239.179993,239.595001,260517
2025-09-05 13:33:00+00:00,240.110992,240.330002,239.839996,239.930099,270253
2025-09-05 13:34:00+00:00,240.070007,240.619995,239.925003,240.139999,597480
...,...,...,...,...,...
2025-09-16 16:06:00+00:00,238.860001,238.889999,238.820007,238.875000,44984
2025-09-16 16:07:00+00:00,238.854996,238.889999,238.850006,238.869995,42225
2025-09-16 16:08:00+00:00,238.850006,238.860001,238.839996,238.860001,36856


In [9]:
from backtesting import Strategy, Backtest

In [ ]:
class OpeningRangeBreakout(Strategy):
    
    def init(self):
        self.opening_range_high = None
        self.opening_range_low = None
        
    def next(self):
        #print(self.data)
        pass
        
bt = Backtest(df, OpeningRangeBreakout, cash=25000)

stats = bt.run()

from bokeh.io import output_notebook
output_notebook()
bt.plot()

Loading BokehJS ...

GridPlot(id='p2064', ...)